## Model Layers

This module contains many layer classes that we might be interested in using in our models. These layers complement the default [Pytorch layers](https://pytorch.org/docs/stable/nn.html) which we can also use as predefined layers.

In [ ]:
from fastai.vision import *
from fastai.gen_doc.nbdoc import *

## Custom fastai modules

In [ ]:
show_doc(AdaptiveConcatPool2d, title_level=3)

<h3 id="AdaptiveConcatPool2d" class="doc_header"><code>class</code> <code>AdaptiveConcatPool2d</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L176" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#AdaptiveConcatPool2d-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>AdaptiveConcatPool2d</code>(**`sz`**:`Optional`\[`int`\]=***`None`***) :: [`PrePostInitMeta`](/core.html#PrePostInitMeta) :: [`Module`](/torch_core.html#Module)

<div class="collapse" id="AdaptiveConcatPool2d-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#AdaptiveConcatPool2d-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>AdaptiveConcatPool2d</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Layer that concats `AdaptiveAvgPool2d` and `AdaptiveMaxPool2d`.  

In [ ]:
from fastai.gen_doc.nbdoc import *
from fastai.layers import * 

The output will be `2*sz`, or just 2 if `sz` is None.

The [`AdaptiveConcatPool2d`](/layers.html#AdaptiveConcatPool2d) object uses adaptive average pooling and adaptive max pooling and concatenates them both. We use this because it provides the model with the information of both methods and improves performance. This technique is called `adaptive` because it allows us to decide on what output dimensions we want, instead of choosing the input's dimensions to fit a desired output size.

Let's try training with Adaptive Average Pooling first, then with Adaptive Max Pooling and finally with the concatenation of them both to see how they fare in performance.

We will first define a [`simple_cnn`](/layers.html#simple_cnn) using [Adaptive Max Pooling](https://pytorch.org/docs/stable/nn.html#torch.nn.AdaptiveMaxPool2d) by changing the source code a bit.

In [ ]:
path = untar_data(URLs.MNIST_SAMPLE)
data = ImageDataBunch.from_folder(path)

In [ ]:
def simple_cnn_max(actns:Collection[int], kernel_szs:Collection[int]=None,
               strides:Collection[int]=None) -> nn.Sequential:
    "CNN with `conv2d_relu` layers defined by `actns`, `kernel_szs` and `strides`"
    nl = len(actns)-1
    kernel_szs = ifnone(kernel_szs, [3]*nl)
    strides    = ifnone(strides   , [2]*nl)
    layers = [conv_layer(actns[i], actns[i+1], kernel_szs[i], stride=strides[i])
        for i in range(len(strides))]
    layers.append(nn.Sequential(nn.AdaptiveMaxPool2d(1), Flatten()))
    return nn.Sequential(*layers)

In [ ]:
model = simple_cnn_max((3,16,16,2))
learner = Learner(data, model, metrics=[accuracy])
learner.fit(1)

epoch,train_loss,valid_loss,accuracy
1,0.102758,0.064676,0.984298


Now let's try with [Adaptive Average Pooling](https://pytorch.org/docs/stable/nn.html#torch.nn.AdaptiveAvgPool2d) now.

In [ ]:
def simple_cnn_avg(actns:Collection[int], kernel_szs:Collection[int]=None,
               strides:Collection[int]=None) -> nn.Sequential:
    "CNN with `conv2d_relu` layers defined by `actns`, `kernel_szs` and `strides`"
    nl = len(actns)-1
    kernel_szs = ifnone(kernel_szs, [3]*nl)
    strides    = ifnone(strides   , [2]*nl)
    layers = [conv_layer(actns[i], actns[i+1], kernel_szs[i], stride=strides[i])
        for i in range(len(strides))]
    layers.append(nn.Sequential(nn.AdaptiveAvgPool2d(1), Flatten()))
    return nn.Sequential(*layers)

In [ ]:
model = simple_cnn_avg((3,16,16,2))
learner = Learner(data, model, metrics=[accuracy])
learner.fit(1)

epoch,train_loss,valid_loss,accuracy
1,0.241485,0.201116,0.973994


Finally we will try with the concatenation of them both [`AdaptiveConcatPool2d`](/layers.html#AdaptiveConcatPool2d). We will see that, in fact, it increases our accuracy and decreases our loss considerably!

In [ ]:
def simple_cnn(actns:Collection[int], kernel_szs:Collection[int]=None,
               strides:Collection[int]=None) -> nn.Sequential:
    "CNN with `conv2d_relu` layers defined by `actns`, `kernel_szs` and `strides`"
    nl = len(actns)-1
    kernel_szs = ifnone(kernel_szs, [3]*nl)
    strides    = ifnone(strides   , [2]*nl)
    layers = [conv_layer(actns[i], actns[i+1], kernel_szs[i], stride=strides[i])
        for i in range(len(strides))]
    layers.append(nn.Sequential(AdaptiveConcatPool2d(1), Flatten()))
    return nn.Sequential(*layers)

In [ ]:
model = simple_cnn((3,16,16,2))
learner = Learner(data, model, metrics=[accuracy])
learner.fit(1)

epoch,train_loss,valid_loss,accuracy
1,0.203015,0.122094,0.988224


In [ ]:
show_doc(Lambda, title_level=3)

<h3 id="Lambda" class="doc_header"><code>class</code> <code>Lambda</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L10" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#Lambda-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>Lambda</code>(**`func`**:`LambdaFunc`) :: [`PrePostInitMeta`](/core.html#PrePostInitMeta) :: [`Module`](/torch_core.html#Module)

<div class="collapse" id="Lambda-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#Lambda-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>Lambda</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Create a layer that simply calls `func` with `x`  

This is very useful to use functions as layers in our networks inside a [Sequential](https://pytorch.org/docs/stable/nn.html#torch.nn.Sequential) object. So, for example, say we want to apply a [log_softmax loss](https://pytorch.org/docs/stable/nn.html#torch.nn.functional.log_softmax) and we need to change the shape of our output batches to be able to use this loss. We can add a layer that applies the necessary change in shape by calling:

`Lambda(lambda x: x.view(x.size(0),-1))`

Let's see an example of how the shape of our output can change when we add this layer.

In [ ]:
model = nn.Sequential(
    nn.Conv2d(3,  16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.AdaptiveAvgPool2d(1),
)

model.cuda()

for xb, yb in data.train_dl:
    out = (model(*[xb]))
    print(out.size())
    break

torch.Size([64, 10, 1, 1])


In [ ]:
model = nn.Sequential(
    nn.Conv2d(3,  16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.AdaptiveAvgPool2d(1),
    Lambda(lambda x: x.view(x.size(0),-1))
)

model.cuda()

for xb, yb in data.train_dl:
    out = (model(*[xb]))
    print(out.size())
    break

torch.Size([64, 10])


In [ ]:
show_doc(Flatten)

<h2 id="Flatten" class="doc_header"><code>class</code> <code>Flatten</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L25" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#Flatten-pytest" style="float:right; padding-right:10px">[test]</a></h2>

> <code>Flatten</code>(**`full`**:`bool`=***`False`***) :: [`PrePostInitMeta`](/core.html#PrePostInitMeta) :: [`Module`](/torch_core.html#Module)

<div class="collapse" id="Flatten-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#Flatten-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>Flatten</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Flatten `x` to a single dimension, often used at the end of a model. `full` for rank-1 tensor  

The function we build above is actually implemented in our library as [`Flatten`](/layers.html#Flatten). We can see that it returns the same size when we run it.

In [ ]:
model = nn.Sequential(
    nn.Conv2d(3,  16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.AdaptiveAvgPool2d(1),
    Flatten(),
)

model.cuda()

for xb, yb in data.train_dl:
    out = (model(*[xb]))
    print(out.size())
    break

torch.Size([64, 10])


In [ ]:
show_doc(PoolFlatten)

<h4 id="PoolFlatten" class="doc_header"><code>PoolFlatten</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L30" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#PoolFlatten-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>PoolFlatten</code>() → [`Sequential`](https://pytorch.org/docs/stable/nn.html#torch.nn.Sequential)

<div class="collapse" id="PoolFlatten-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#PoolFlatten-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>PoolFlatten</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Apply [`nn.AdaptiveAvgPool2d`](https://pytorch.org/docs/stable/nn.html#torch.nn.AdaptiveAvgPool2d) to `x` and then flatten the result.  

We can combine these two final layers ([AdaptiveAvgPool2d](https://pytorch.org/docs/stable/nn.html#torch.nn.AdaptiveAvgPool2d) and [`Flatten`](/layers.html#Flatten)) by using [`PoolFlatten`](/layers.html#PoolFlatten).

In [ ]:
model = nn.Sequential(
    nn.Conv2d(3,  16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    PoolFlatten()
)

model.cuda()

for xb, yb in data.train_dl:
    out = (model(*[xb]))
    print(out.size())
    break

torch.Size([64, 10])


Another use we give to the Lambda function is to resize batches with [`ResizeBatch`](/layers.html#ResizeBatch) when we have a layer that expects a different input than what comes from the previous one.

In [ ]:
show_doc(ResizeBatch)

<h2 id="ResizeBatch" class="doc_header"><code>class</code> <code>ResizeBatch</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L20" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#ResizeBatch-pytest" style="float:right; padding-right:10px">[test]</a></h2>

> <code>ResizeBatch</code>(**\*`size`**:`int`) :: [`PrePostInitMeta`](/core.html#PrePostInitMeta) :: [`Module`](/torch_core.html#Module)

<div class="collapse" id="ResizeBatch-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#ResizeBatch-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>ResizeBatch</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Reshape `x` to `size`, keeping batch dim the same size  

In [ ]:
a = torch.tensor([[1., -1.], [1., -1.]])[None]
print(a)

tensor([[[ 1., -1.],
         [ 1., -1.]]])


In [ ]:
out = ResizeBatch(4)
print(out(a))

tensor([[ 1., -1.,  1., -1.]])


In [ ]:
show_doc(Debugger, title_level=3)

<h3 id="Debugger" class="doc_header"><code>class</code> <code>Debugger</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L186" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#Debugger-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>Debugger</code>() :: [`PrePostInitMeta`](/core.html#PrePostInitMeta) :: [`Module`](/torch_core.html#Module)

<div class="collapse" id="Debugger-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#Debugger-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>Debugger</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

A module to debug inside a model.  

The debugger module allows us to peek inside a network while its training and see in detail what is going on. We can see inputs, outputs and sizes at any point in the network.

For instance, if you run the following:

``` python
model = nn.Sequential(
    nn.Conv2d(3,  16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    Debugger(),
    nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1), nn.ReLU(),
)

model.cuda()

learner = Learner(data, model, metrics=[accuracy])
learner.fit(5)
```
... you'll see something like this:

```
/home/ubuntu/fastai/fastai/layers.py(74)forward()
     72     def forward(self,x:Tensor) -> Tensor:
     73         set_trace()
---> 74         return x
     75 
     76 class StdUpsample(nn.Module):

ipdb>
```

In [ ]:
show_doc(PixelShuffle_ICNR, title_level=3)

<h3 id="PixelShuffle_ICNR" class="doc_header"><code>class</code> <code>PixelShuffle_ICNR</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L202" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#PixelShuffle_ICNR-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>PixelShuffle_ICNR</code>(**`ni`**:`int`, **`nf`**:`int`=***`None`***, **`scale`**:`int`=***`2`***, **`blur`**:`bool`=***`False`***, **`norm_type`**=***`<NormType.Weight: 3>`***, **`leaky`**:`float`=***`None`***) :: [`PrePostInitMeta`](/core.html#PrePostInitMeta) :: [`Module`](/torch_core.html#Module)

<div class="collapse" id="PixelShuffle_ICNR-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#PixelShuffle_ICNR-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>PixelShuffle_ICNR</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Upsample by `scale` from `ni` filters to `nf` (default `ni`), using [`nn.PixelShuffle`](https://pytorch.org/docs/stable/nn.html#torch.nn.PixelShuffle), [`icnr`](/layers.html#icnr) init, and [`weight_norm`](https://pytorch.org/docs/stable/nn.html#torch.nn.weight_norm).  

In [ ]:
show_doc(MergeLayer, title_level=3)

<h3 id="MergeLayer" class="doc_header"><code>class</code> <code>MergeLayer</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L147" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#MergeLayer-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>MergeLayer</code>(**`dense`**:`bool`=***`False`***) :: [`PrePostInitMeta`](/core.html#PrePostInitMeta) :: [`Module`](/torch_core.html#Module)

<div class="collapse" id="MergeLayer-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#MergeLayer-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>MergeLayer</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Merge a shortcut with the result of the module by adding them or concatenating them if `dense=True`.  

In [ ]:
show_doc(PartialLayer, title_level=3)

<h3 id="PartialLayer" class="doc_header"><code>class</code> <code>PartialLayer</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L170" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#PartialLayer-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>PartialLayer</code>(**`func`**, **\*\*`kwargs`**) :: [`PrePostInitMeta`](/core.html#PrePostInitMeta) :: [`Module`](/torch_core.html#Module)

<div class="collapse" id="PartialLayer-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#PartialLayer-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>PartialLayer</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Layer that applies `partial(func, **kwargs)`.  

In [ ]:
show_doc(SigmoidRange, title_level=3)

<h3 id="SigmoidRange" class="doc_header"><code>class</code> <code>SigmoidRange</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L165" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#SigmoidRange-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>SigmoidRange</code>(**`low`**, **`high`**) :: [`PrePostInitMeta`](/core.html#PrePostInitMeta) :: [`Module`](/torch_core.html#Module)

<div class="collapse" id="SigmoidRange-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#SigmoidRange-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>SigmoidRange</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Sigmoid module with range `(low,x_max)`  

In [ ]:
show_doc(SequentialEx, title_level=3)

<h3 id="SequentialEx" class="doc_header"><code>class</code> <code>SequentialEx</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L128" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#SequentialEx-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>SequentialEx</code>(**\*`layers`**) :: [`PrePostInitMeta`](/core.html#PrePostInitMeta) :: [`Module`](/torch_core.html#Module)

<div class="collapse" id="SequentialEx-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#SequentialEx-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>SequentialEx</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Like [`nn.Sequential`](https://pytorch.org/docs/stable/nn.html#torch.nn.Sequential), but with ModuleList semantics, and can access module input  

In [ ]:
show_doc(SelfAttention, title_level=3)

<h3 id="SelfAttention" class="doc_header"><code>class</code> <code>SelfAttention</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L81" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#SelfAttention-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>SelfAttention</code>(**`n_channels`**:`int`) :: [`PrePostInitMeta`](/core.html#PrePostInitMeta) :: [`Module`](/torch_core.html#Module)

<div class="collapse" id="SelfAttention-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#SelfAttention-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>Tests found for <code>SelfAttention</code>:</p><ul><li><code>pytest -sv tests/test_torch_core.py::test_keep_parameter</code> <a href="https://github.com/fastai/fastai/blob/master/tests/test_torch_core.py#L269" class="source_link" style="float:right">[source]</a></li></ul><p>To run tests please refer to this <a href="/dev/test.html#quick-guide">guide</a>.</p></div></div>

Self attention layer for nd.  

In [ ]:
show_doc(BatchNorm1dFlat, title_level=3)

<h3 id="BatchNorm1dFlat" class="doc_header"><code>class</code> <code>BatchNorm1dFlat</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L292" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#BatchNorm1dFlat-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>BatchNorm1dFlat</code>(**`num_features`**, **`eps`**=***`1e-05`***, **`momentum`**=***`0.1`***, **`affine`**=***`True`***, **`track_running_stats`**=***`True`***) :: [`BatchNorm1d`](https://pytorch.org/docs/stable/nn.html#torch.nn.BatchNorm1d)

<div class="collapse" id="BatchNorm1dFlat-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#BatchNorm1dFlat-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>BatchNorm1dFlat</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

[`nn.BatchNorm1d`](https://pytorch.org/docs/stable/nn.html#torch.nn.BatchNorm1d), but first flattens leading dimensions  

## Loss functions

In [ ]:
show_doc(FlattenedLoss, title_level=3)

<h3 id="FlattenedLoss" class="doc_header"><code>class</code> <code>FlattenedLoss</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L221" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#FlattenedLoss-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>FlattenedLoss</code>(**`func`**, **\*`args`**, **`axis`**:`int`=***`-1`***, **`floatify`**:`bool`=***`False`***, **`is_2d`**:`bool`=***`True`***, **\*\*`kwargs`**)

<div class="collapse" id="FlattenedLoss-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#FlattenedLoss-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>FlattenedLoss</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Same as `func`, but flattens input and target.  

Create an instance of `func` with `args` and `kwargs`. When passing an output and target, it
- puts `axis` first in output and target with a transpose
- casts the target to `float` if `floatify=True`
- squeezes the `output` to two dimensions if `is_2d`, otherwise one dimension, squeezes the target to one dimension
- applies the instance of `func`.

In [ ]:
show_doc(BCEFlat)

<h4 id="BCEFlat" class="doc_header"><code>BCEFlat</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L253" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#BCEFlat-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>BCEFlat</code>(**\*`args`**, **`axis`**:`int`=***`-1`***, **`floatify`**:`bool`=***`True`***, **\*\*`kwargs`**)

<div class="collapse" id="BCEFlat-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#BCEFlat-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>BCEFlat</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Same as [`nn.BCELoss`](https://pytorch.org/docs/stable/nn.html#torch.nn.BCELoss), but flattens input and target.  

In [ ]:
show_doc(BCEWithLogitsFlat)

<h4 id="BCEWithLogitsFlat" class="doc_header"><code>BCEWithLogitsFlat</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L249" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#BCEWithLogitsFlat-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>BCEWithLogitsFlat</code>(**\*`args`**, **`axis`**:`int`=***`-1`***, **`floatify`**:`bool`=***`True`***, **\*\*`kwargs`**)

<div class="collapse" id="BCEWithLogitsFlat-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#BCEWithLogitsFlat-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>BCEWithLogitsFlat</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Same as [`nn.BCEWithLogitsLoss`](https://pytorch.org/docs/stable/nn.html#torch.nn.BCEWithLogitsLoss), but flattens input and target.  

In [ ]:
show_doc(CrossEntropyFlat)

<h4 id="CrossEntropyFlat" class="doc_header"><code>CrossEntropyFlat</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L245" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#CrossEntropyFlat-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>CrossEntropyFlat</code>(**\*`args`**, **`axis`**:`int`=***`-1`***, **\*\*`kwargs`**)

<div class="collapse" id="CrossEntropyFlat-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#CrossEntropyFlat-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>CrossEntropyFlat</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Same as [`nn.CrossEntropyLoss`](https://pytorch.org/docs/stable/nn.html#torch.nn.CrossEntropyLoss), but flattens input and target.  

In [ ]:
show_doc(MSELossFlat)

<h4 id="MSELossFlat" class="doc_header"><code>MSELossFlat</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L257" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#MSELossFlat-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>MSELossFlat</code>(**\*`args`**, **`axis`**:`int`=***`-1`***, **`floatify`**:`bool`=***`True`***, **\*\*`kwargs`**)

<div class="collapse" id="MSELossFlat-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#MSELossFlat-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>MSELossFlat</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Same as [`nn.MSELoss`](https://pytorch.org/docs/stable/nn.html#torch.nn.MSELoss), but flattens input and target.  

In [ ]:
show_doc(NoopLoss)

<h2 id="NoopLoss" class="doc_header"><code>class</code> <code>NoopLoss</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L261" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#NoopLoss-pytest" style="float:right; padding-right:10px">[test]</a></h2>

> <code>NoopLoss</code>() :: [`PrePostInitMeta`](/core.html#PrePostInitMeta) :: [`Module`](/torch_core.html#Module)

<div class="collapse" id="NoopLoss-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#NoopLoss-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>NoopLoss</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Just returns the mean of the `output`.  

In [ ]:
show_doc(WassersteinLoss)

<h2 id="WassersteinLoss" class="doc_header"><code>class</code> <code>WassersteinLoss</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L265" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#WassersteinLoss-pytest" style="float:right; padding-right:10px">[test]</a></h2>

> <code>WassersteinLoss</code>() :: [`PrePostInitMeta`](/core.html#PrePostInitMeta) :: [`Module`](/torch_core.html#Module)

<div class="collapse" id="WassersteinLoss-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#WassersteinLoss-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>WassersteinLoss</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

For WGAN.  

## Helper functions to create modules

In [ ]:
show_doc(bn_drop_lin, doc_string=False)

<h4 id="bn_drop_lin" class="doc_header"><code>bn_drop_lin</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L44" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#bn_drop_lin-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>bn_drop_lin</code>(**`n_in`**:`int`, **`n_out`**:`int`, **`bn`**:`bool`=***`True`***, **`p`**:`float`=***`0.0`***, **`actn`**:`Optional`\[[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)\]=***`None`***)

<div class="collapse" id="bn_drop_lin-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#bn_drop_lin-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>bn_drop_lin</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

The [`bn_drop_lin`](/layers.html#bn_drop_lin) function returns a sequence of [batch normalization](https://arxiv.org/abs/1502.03167), [dropout](https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf) and a linear layer. This custom layer is usually used at the end of a model. 

`n_in` represents the size of the input, `n_out` the size of the output, `bn` whether we want batch norm or not, `p` how much dropout, and `actn` (optional parameter) adds an activation function at the end.

In [ ]:
show_doc(conv2d)

<h4 id="conv2d" class="doc_header"><code>conv2d</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L98" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#conv2d-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>conv2d</code>(**`ni`**:`int`, **`nf`**:`int`, **`ks`**:`int`=***`3`***, **`stride`**:`int`=***`1`***, **`padding`**:`int`=***`None`***, **`bias`**=***`False`***, **`init`**:`LayerFunc`=***`'kaiming_normal_'`***) → [`Conv2d`](https://pytorch.org/docs/stable/nn.html#torch.nn.Conv2d)

<div class="collapse" id="conv2d-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#conv2d-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>conv2d</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Create and initialize [`nn.Conv2d`](https://pytorch.org/docs/stable/nn.html#torch.nn.Conv2d) layer. `padding` defaults to `ks//2`.  

In [ ]:
show_doc(conv2d_trans)

<h4 id="conv2d_trans" class="doc_header"><code>conv2d_trans</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L103" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#conv2d_trans-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>conv2d_trans</code>(**`ni`**:`int`, **`nf`**:`int`, **`ks`**:`int`=***`2`***, **`stride`**:`int`=***`2`***, **`padding`**:`int`=***`0`***, **`bias`**=***`False`***) → [`ConvTranspose2d`](https://pytorch.org/docs/stable/nn.html#torch.nn.ConvTranspose2d)

<div class="collapse" id="conv2d_trans-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#conv2d_trans-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>conv2d_trans</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Create [`nn.ConvTranspose2d`](https://pytorch.org/docs/stable/nn.html#torch.nn.ConvTranspose2d) layer.  

In [ ]:
show_doc(conv_layer, doc_string=False)

<h4 id="conv_layer" class="doc_header"><code>conv_layer</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L111" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#conv_layer-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>conv_layer</code>(**`ni`**:`int`, **`nf`**:`int`, **`ks`**:`int`=***`3`***, **`stride`**:`int`=***`1`***, **`padding`**:`int`=***`None`***, **`bias`**:`bool`=***`None`***, **`is_1d`**:`bool`=***`False`***, **`norm_type`**:`Optional`\[[`NormType`](/layers.html#NormType)\]=***`<NormType.Batch: 1>`***, **`use_activ`**:`bool`=***`True`***, **`leaky`**:`float`=***`None`***, **`transpose`**:`bool`=***`False`***, **`init`**:`Callable`=***`'kaiming_normal_'`***, **`self_attention`**:`bool`=***`False`***)

<div class="collapse" id="conv_layer-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#conv_layer-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>conv_layer</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

The [`conv_layer`](/layers.html#conv_layer) function returns a sequence of [nn.Conv2D](https://pytorch.org/docs/stable/nn.html#torch.nn.Conv2d), [BatchNorm](https://arxiv.org/abs/1502.03167) and a ReLU or [leaky RELU](https://ai.stanford.edu/~amaas/papers/relu_hybrid_icml2013_final.pdf) activation function.

`n_in` represents the size of the input, `n_out` the size of the output, `ks` the kernel size, `stride` the stride with which we want to apply the convolutions. `bias` will decide if they have bias or not (if None, defaults to True unless using batchnorm). `norm_type` selects the type of normalization (or `None`). If `leaky` is None, the activation is a standard `ReLU`, otherwise it's a `LeakyReLU` of slope `leaky`. Finally if `transpose=True`, the convolution is replaced by a `ConvTranspose2D`.

In [ ]:
show_doc(embedding, doc_string=False)

<h4 id="embedding" class="doc_header"><code>embedding</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L285" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#embedding-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>embedding</code>(**`ni`**:`int`, **`nf`**:`int`) → [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)

<div class="collapse" id="embedding-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#embedding-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>embedding</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Create an [embedding layer](https://arxiv.org/abs/1711.09160) with input size `ni` and output size `nf`.

In [ ]:
show_doc(relu)

<h4 id="relu" class="doc_header"><code>relu</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L107" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#relu-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>relu</code>(**`inplace`**:`bool`=***`False`***, **`leaky`**:`float`=***`None`***)

<div class="collapse" id="relu-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#relu-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>relu</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Return a relu activation, maybe `leaky` and `inplace`.  

In [ ]:
show_doc(res_block)

<h4 id="res_block" class="doc_header"><code>res_block</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L152" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#res_block-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>res_block</code>(**`nf`**, **`dense`**:`bool`=***`False`***, **`norm_type`**:`Optional`\[[`NormType`](/layers.html#NormType)\]=***`<NormType.Batch: 1>`***, **`bottle`**:`bool`=***`False`***, **\*\*`conv_kwargs`**)

<div class="collapse" id="res_block-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#res_block-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>res_block</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Resnet block of `nf` features. `conv_kwargs` are passed to [`conv_layer`](/layers.html#conv_layer).  

In [ ]:
show_doc(sigmoid_range)

<h4 id="sigmoid_range" class="doc_header"><code>sigmoid_range</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L161" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#sigmoid_range-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>sigmoid_range</code>(**`x`**, **`low`**, **`high`**)

<div class="collapse" id="sigmoid_range-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#sigmoid_range-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>sigmoid_range</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Sigmoid function with range `(low, high)`  

In [ ]:
show_doc(simple_cnn)

<h4 id="simple_cnn" class="doc_header"><code>simple_cnn</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L269" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#simple_cnn-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>simple_cnn</code>(**`actns`**:`Collection`\[`int`\], **`kernel_szs`**:`Collection`\[`int`\]=***`None`***, **`strides`**:`Collection`\[`int`\]=***`None`***, **`bn`**=***`False`***) → [`Sequential`](https://pytorch.org/docs/stable/nn.html#torch.nn.Sequential)

<div class="collapse" id="simple_cnn-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#simple_cnn-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>simple_cnn</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

CNN with [`conv_layer`](/layers.html#conv_layer) defined by `actns`, `kernel_szs` and `strides`, plus batchnorm if `bn`.  

## Initialization of modules

In [ ]:
show_doc(batchnorm_2d)

<h4 id="batchnorm_2d" class="doc_header"><code>batchnorm_2d</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L36" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#batchnorm_2d-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>batchnorm_2d</code>(**`nf`**:`int`, **`norm_type`**:[`NormType`](/layers.html#NormType)=***`<NormType.Batch: 1>`***)

<div class="collapse" id="batchnorm_2d-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#batchnorm_2d-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>batchnorm_2d</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

A batchnorm2d layer with `nf` features initialized depending on `norm_type`.  

In [ ]:
show_doc(icnr)

<h4 id="icnr" class="doc_header"><code>icnr</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L192" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#icnr-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>icnr</code>(**`x`**, **`scale`**=***`2`***, **`init`**=***`'kaiming_normal_'`***)

<div class="collapse" id="icnr-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#icnr-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>icnr</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

ICNR init of `x`, with `scale` and `init` function.  

In [ ]:
show_doc(trunc_normal_)

<h4 id="trunc_normal_" class="doc_header"><code>trunc_normal_</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L280" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#trunc_normal_-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>trunc_normal_</code>(**`x`**:`Tensor`, **`mean`**:`float`=***`0.0`***, **`std`**:`float`=***`1.0`***) → `Tensor`

<div class="collapse" id="trunc_normal_-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#trunc_normal_-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>trunc_normal_</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Truncated normal initialization.  

In [ ]:
show_doc(icnr)

<h4 id="icnr" class="doc_header"><code>icnr</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L192" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#icnr-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>icnr</code>(**`x`**, **`scale`**=***`2`***, **`init`**=***`'kaiming_normal_'`***)

<div class="collapse" id="icnr-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#icnr-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>icnr</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

ICNR init of `x`, with `scale` and `init` function.  

In [ ]:
show_doc(NormType)

<h2 id="NormType" class="doc_header">`NormType`<a class="source_link" data-toggle="collapse" data-target="#NormType-pytest" style="float:right; padding-right:10px">[test]</a></h2>

> <code>Enum</code> = [Batch, BatchZero, Weight, Spectral]

<div class="collapse" id="NormType-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#NormType-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>NormType</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

An enumeration.  

## Undocumented Methods - Methods moved below this line will intentionally be hidden

In [ ]:
show_doc(Debugger.forward)

<h4 id="Debugger.forward" class="doc_header"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L188" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#Debugger-forward-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>forward</code>(**`x`**:`Tensor`) → `Tensor`

<div class="collapse" id="Debugger-forward-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#Debugger-forward-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>forward</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:[`Module`](/torch_core.html#Module) instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [ ]:
show_doc(Lambda.forward)

<h4 id="Lambda.forward" class="doc_header"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L13" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#Lambda-forward-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>forward</code>(**`x`**)

<div class="collapse" id="Lambda-forward-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#Lambda-forward-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>forward</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:[`Module`](/torch_core.html#Module) instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [ ]:
show_doc(AdaptiveConcatPool2d.forward)

<h4 id="AdaptiveConcatPool2d.forward" class="doc_header"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L184" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#AdaptiveConcatPool2d-forward-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>forward</code>(**`x`**)

<div class="collapse" id="AdaptiveConcatPool2d-forward-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#AdaptiveConcatPool2d-forward-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>forward</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:[`Module`](/torch_core.html#Module) instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [ ]:
show_doc(NoopLoss.forward)

<h4 id="NoopLoss.forward" class="doc_header"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L263" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#NoopLoss-forward-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>forward</code>(**`output`**, **\*`args`**)

<div class="collapse" id="NoopLoss-forward-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#NoopLoss-forward-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>forward</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:[`Module`](/torch_core.html#Module) instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [ ]:
show_doc(PixelShuffle_ICNR.forward)

<h4 id="PixelShuffle_ICNR.forward" class="doc_header"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L217" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#PixelShuffle_ICNR-forward-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>forward</code>(**`x`**)

<div class="collapse" id="PixelShuffle_ICNR-forward-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#PixelShuffle_ICNR-forward-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>forward</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:[`Module`](/torch_core.html#Module) instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [ ]:
show_doc(WassersteinLoss.forward)

<h4 id="WassersteinLoss.forward" class="doc_header"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L267" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#WassersteinLoss-forward-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>forward</code>(**`real`**, **`fake`**)

<div class="collapse" id="WassersteinLoss-forward-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#WassersteinLoss-forward-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>forward</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:[`Module`](/torch_core.html#Module) instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [ ]:
show_doc(MergeLayer.forward)

<h4 id="MergeLayer.forward" class="doc_header"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L150" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#MergeLayer-forward-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>forward</code>(**`x`**)

<div class="collapse" id="MergeLayer-forward-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#MergeLayer-forward-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>forward</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:[`Module`](/torch_core.html#Module) instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [ ]:
show_doc(SigmoidRange.forward)

<h4 id="SigmoidRange.forward" class="doc_header"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L168" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#SigmoidRange-forward-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>forward</code>(**`x`**)

<div class="collapse" id="SigmoidRange-forward-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#SigmoidRange-forward-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>forward</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:[`Module`](/torch_core.html#Module) instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [ ]:
show_doc(MergeLayer.forward)

<h4 id="MergeLayer.forward" class="doc_header"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L150" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#MergeLayer-forward-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>forward</code>(**`x`**)

<div class="collapse" id="MergeLayer-forward-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#MergeLayer-forward-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>forward</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:[`Module`](/torch_core.html#Module) instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [ ]:
show_doc(SelfAttention.forward)

<h4 id="SelfAttention.forward" class="doc_header"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L89" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#SelfAttention-forward-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>forward</code>(**`x`**)

<div class="collapse" id="SelfAttention-forward-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#SelfAttention-forward-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>forward</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:[`Module`](/torch_core.html#Module) instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [ ]:
show_doc(SequentialEx.forward)

<h4 id="SequentialEx.forward" class="doc_header"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L132" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#SequentialEx-forward-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>forward</code>(**`x`**)

<div class="collapse" id="SequentialEx-forward-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#SequentialEx-forward-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>forward</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:[`Module`](/torch_core.html#Module) instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [ ]:
show_doc(SequentialEx.append)

<h4 id="SequentialEx.append" class="doc_header"><code>append</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L143" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#SequentialEx-append-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>append</code>(**`l`**)

<div class="collapse" id="SequentialEx-append-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#SequentialEx-append-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>append</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

In [ ]:
show_doc(SequentialEx.extend)

<h4 id="SequentialEx.extend" class="doc_header"><code>extend</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L144" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#SequentialEx-extend-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>extend</code>(**`l`**)

<div class="collapse" id="SequentialEx-extend-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#SequentialEx-extend-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>extend</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

In [ ]:
show_doc(SequentialEx.insert)

<h4 id="SequentialEx.insert" class="doc_header"><code>insert</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L145" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#SequentialEx-insert-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>insert</code>(**`i`**, **`l`**)

<div class="collapse" id="SequentialEx-insert-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#SequentialEx-insert-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>insert</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

In [ ]:
show_doc(PartialLayer.forward)

<h4 id="PartialLayer.forward" class="doc_header"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L173" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#PartialLayer-forward-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>forward</code>(**`x`**)

<div class="collapse" id="PartialLayer-forward-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#PartialLayer-forward-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>forward</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:[`Module`](/torch_core.html#Module) instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [ ]:
show_doc(BatchNorm1dFlat.forward)

<h4 id="BatchNorm1dFlat.forward" class="doc_header"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L294" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#BatchNorm1dFlat-forward-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>forward</code>(**`x`**)

<div class="collapse" id="BatchNorm1dFlat-forward-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#BatchNorm1dFlat-forward-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>forward</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:[`Module`](/torch_core.html#Module) instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [ ]:
show_doc(Flatten.forward)

<h4 id="Flatten.forward" class="doc_header"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L28" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#Flatten-forward-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>forward</code>(**`x`**)

<div class="collapse" id="Flatten-forward-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#Flatten-forward-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>forward</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:[`Module`](/torch_core.html#Module) instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

## New Methods - Please document or move to the undocumented section

In [ ]:
show_doc(View)

<h2 id="View" class="doc_header"><code>class</code> <code>View</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L15" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#View-pytest" style="float:right; padding-right:10px">[test]</a></h2>

> <code>View</code>(**\*`size`**:`int`) :: [`PrePostInitMeta`](/core.html#PrePostInitMeta) :: [`Module`](/torch_core.html#Module)

<div class="collapse" id="View-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#View-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>View</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Reshape `x` to `size`  

In [ ]:
show_doc(ResizeBatch.forward)

<h4 id="ResizeBatch.forward" class="doc_header"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L23" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#ResizeBatch-forward-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>forward</code>(**`x`**)

<div class="collapse" id="ResizeBatch-forward-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#ResizeBatch-forward-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>forward</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:[`Module`](/torch_core.html#Module) instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [ ]:
show_doc(View.forward)

<h4 id="View.forward" class="doc_header"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L18" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#View-forward-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>forward</code>(**`x`**)

<div class="collapse" id="View-forward-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#View-forward-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>forward</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:[`Module`](/torch_core.html#Module) instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 